In [1]:
import sqlite3
import pandas as pd
import re

In [2]:
# connect to or create database

conn = sqlite3.connect("paws.db")

In [3]:
# function for loading csvs into database tables

def load_to_sqlite(csv_name, table_name, connection):
    
    # load csv into a dataframe
    df = pd.read_csv(csv_name, encoding='cp1252')
    
    # strip whitespace and periods from headers, convert to lowercase
    df.columns = df.columns.str.lower().str.strip()
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.map(lambda x: re.sub(r'\.+', '_', x))
    
    # drop dataframe into database table
    df.to_sql(table_name, connection, index=False,)

In [4]:
# load petpoint

load_to_sqlite('CfP_PDP_petpoint_deidentified.csv', 'petpoint', conn)

In [5]:
# load volgistics

load_to_sqlite('CfP_PDP_volgistics_deidentified.csv', 'volgistics', conn)

In [6]:
# load salesforce contacts

load_to_sqlite('CfP_PDP_salesforceContacts_deidentified.csv', 'salesforcecontacts', conn)

In [7]:
# load salesforce donations

load_to_sqlite('CfP_PDP_salesforceDonations_deidentified.csv', 'salesforcedonations', conn)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [8]:
# simple join to check that it worked and the tables can be queried

df = pd.read_sql('''select * from petpoint as pp 
                    join volgistics as vol 
                    on pp."unnamed:_0" = vol."unnamed:_0"

                    join (SELECT * FROM salesforcecontacts AS sf_contacts
                            JOIN salesforcedonations AS sf_donations
                            ON sf_contacts."Account_ID" = sf_donations."Account_ID") as sf
                    on pp."unnamed:_0" = sf."unnamed:_0"
                    
                    ''', conn)

df.head()

,unnamed:_0,animal_id,arn,animal_name,animal_type,species,primary_breed,secondary_breed,distinguishing_markings,gender,...,fiscal_period,amount,probability_,age,close_date,created_date,next_step,lead_source,type,primary_campaign_source
0,1,A37324077,None,Wendy Willow Puffs,Cat,Cat,Domestic Shorthair,Mix,None,F,...,Q3-2017,25.0,100,0,8/23/2017,8/23/2017,None,None,One-time,None
1,3,A37498706,None,Niles,Cat,Cat,Domestic Shorthair,Mix,None,M,...,Q3-2017,100.0,100,0,8/9/2017,8/23/2017,None,None,One-time,Mutt Strut 2016 Donations
2,3,A37498706,None,Niles,Cat,Cat,Domestic Shorthair,Mix,None,M,...,Q4-2015,10.0,100,0,12/14/2015,12/14/2015,None,None,None,None
3,3,A37498706,None,Niles,Cat,Cat,Domestic Shorthair,Mix,None,M,...,Q3-2019,104.0,100,0,9/23/2019,9/23/2019,None,None,One-time,Monthly
4,3,A37498706,None,Niles,Cat,Cat,Domestic Shorthair,Mix,None,M,...,Q3-2018,22.0,100,0,9/14/2018,9/14/2018,None,None,One-time,PAWS: Monthly Giving


In [9]:
# close database connection

conn.close()